In [1]:
import logging
from functions import alert_logic, gsheets, email, strava_api
import pandas as pd
from dotenv import load_dotenv
import os

# ------------------- Setup Logging -------------------
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
)

logger = logging.getLogger(__name__)

# ------------------- Load Environment -------------------
load_dotenv()
GOOGLE_SHEETS_URL_KEY = os.environ["GOOGLE_SHEETS_URL_KEY"]
GMAIL = os.environ["GMAIL"]
GMAIL_APP_PASSWORD = os.environ["GMAIL_APP_PASSWORD"]



logger.info("Environment variables loaded.")

# ------------------- Get Activities -------------------
try:
    logger.info("Reading existing activities from Google Sheets...")
    sheets_df = gsheets.read_from_gsheets("Activities", GOOGLE_SHEETS_URL_KEY)
    sheets_df["start_date_local"] = pd.to_datetime(sheets_df["start_date_local"])
    logger.info(f"Loaded {len(sheets_df)} existing activities.")

    last_date = sheets_df["start_date_local"].max()
    logger.info(f"Getting new activities since {last_date}...")
    new_activities = strava_api.get_activities_since_date(last_date)
    new_activities_df = strava_api.build_activities_dataframe(new_activities)
    logger.info(f"Retrieved {len(new_activities_df)} new activities.")

    # Combine and deduplicate
    combined_df = pd.concat([sheets_df, new_activities_df], ignore_index=True)
    combined_df = combined_df.drop_duplicates(subset="upload_id", keep="last")
    combined_df = combined_df.sort_values(by="upload_id").reset_index(drop=True)
    logger.info(
        f"Combined dataframe now has {len(combined_df)} rows after deduplication."
    )

    # Write back to Google Sheets
    gsheets.write_to_gsheets(combined_df, "Activities", GOOGLE_SHEETS_URL_KEY)
    logger.info("Updated Activities sheet successfully.")

except Exception as e:
    logger.exception(f"Error fetching or updating activities: {e}")

# ------------------- Get Form Responses -------------------
try:
    logger.info("Reading form responses from Google Sheets...")
    form_responses_df = gsheets.read_from_gsheets(
        "Form Responses", GOOGLE_SHEETS_URL_KEY
    )
    form_responses_df["datetime"] = pd.to_datetime(
        form_responses_df["Date of Action"] + " " + form_responses_df["Time of Action"],
        format="%m/%d/%Y %I:%M:%S %p",
    )
    logger.info(f"Loaded {len(form_responses_df)} form responses.")
except Exception as e:
    logger.exception(f"Error reading form responses: {e}")


2025-10-30 19:23:23 [INFO] Environment variables loaded.
2025-10-30 19:23:23 [INFO] Reading existing activities from Google Sheets...
2025-10-30 19:23:24 [INFO] Loaded 955 existing activities.
2025-10-30 19:23:24 [INFO] Getting new activities since 2025-10-28 17:36:18+00:00...
2025-10-30 19:23:25 [INFO] Retrieved 1 new activities.
2025-10-30 19:23:25 [INFO] Combined dataframe now has 268 rows after deduplication.
2025-10-30 19:23:27 [INFO] Updated Activities sheet successfully.
2025-10-30 19:23:27 [INFO] Reading form responses from Google Sheets...
2025-10-30 19:23:28 [INFO] Loaded 6 form responses.


In [2]:
from functions import alert_logic

alert_df = alert_logic.maintenance_alert(combined_df, form_responses_df, gear_id="b14816258")

alert_df

,issue_alert,date,action_type,miles_threshold,days_threshold,miles_since_last_action,days_since_last_action
0,True,2025-10-30 19:23:28,Wax chain,2.0,NaN,171.857718,14
1,True,2025-10-30 19:23:28,Charge DI2,3.0,90.0,621.269962,59
2,True,2025-10-30 19:23:28,Refill Sealant,NaN,3.0,54.304000,5
